In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import spacy as sp
import json
import matplotlib.pyplot as plt
from collections import Counter

In [4]:
rdata = pd.read_json('../../data/relevant_news_10K.json')
irdata = pd.read_json('../../data/raw/irrelevant_news_10K.json')

ValueError: Expected object or value

In [8]:
rdata.head()

,uid,source,headline,summary,content
0,a437ff48-104a-54bb-bff7-c7a736158524,Agence France Presse,Milkshakes become weapon of choice in UK Europ...,Former UK Independence Party leader Nigel Fara...,The lowly milkshake has turned into an unlikel...
1,366c92af-8143-5ffa-8702-4f26bd22c8b6,News Ltd.,ANZ's first assault in the looming job armageddon,ANZ has moved to the front of the race that’ s...,ANZ has moved to the front of the race that’s ...
2,863096d4-48f0-5a7c-bee6-384a76d575ee,Tribune Content Agency,Carnival Cruise Line to collect your used soap...,Jul. 10-- Carnival Cruise Line stateroom atten...,Jul. 10--Carnival Cruise Line stateroom attend...
3,3e4d6490-4224-595e-be26-4cb249209b8f,SyndiGate Media Inc.,Standard CC bags fourth title in a row [New In...,CHENNAI: R Rohit and P Saravanan took five wic...,CHENNAI: R Rohit (5/43) and P Saravanan (5/46)...
4,9f3e248d-b040-5058-bdc9-61c4de59f02a,Guardian,Revealed: Trump's Wildlife Service pick has ti...,New revelations show she also has ties to the ...,Donald Trump’s nominee to lead the US Fish and...


In [5]:
nlp = sp.load('en_core_web_sm')

In [14]:
def generate_hash(text):
    hash_df = {}
    doc = nlp(text)
    for entity in doc.ents:
        if entity.label_ not in hash_df:
            hash_df[entity.label_] = 0
        hash_df[entity.label_] = hash_df[entity.label_] + 1
    return hash_df

In [49]:
def generate_ner_vector(data, feature):
    '''
    Input: dataframe of news article data, column to perform function
    Output: dataframe of articles with entities as columns and counts of each entity as values
    Notes: column entity definitions: https://spacy.io/api/annotation#named-entities
    '''
    columns = ['uid', 'PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', \
               'LAW', 'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORGINAL', 'CARDINAL']
    df = pd.DataFrame(index=data.index, columns=columns)
    df['uid'] = data['uid']
    for index, row in data.iterrows():
        if (index % 1000 == 0):
            print('progress: ', index/10000)
        text = row[feature]
        hash_df = generate_hash(text)
        itercol = iter(columns)
        next(itercol)
        for col in itercol:
            if col in hash_df:
                df.at[index, col] = hash_df[col]
            else:
                df.at[index, col] = 0
    # diagnostics
    # print(df)
    return df

In [50]:
# Generate dataframe with rows of articles, and columns of entities; cells are counts of each entity in the article content
# Test Case: rdata_ner_df = generate_ner_vector(rdata.loc[[0]], "content")

rdata_ner_df = generate_ner_vector(rdata, "content")

progress:  0.0
progress:  0.1
progress:  0.2
progress:  0.3
progress:  0.4
progress:  0.5
progress:  0.6
progress:  0.7
progress:  0.8
progress:  0.9


In [52]:
irdata_ner_df = generate_ner_vector(irdata, "content")

progress:  0.0
progress:  0.1
progress:  0.2
progress:  0.3
progress:  0.4
progress:  0.5
progress:  0.6
progress:  0.7
progress:  0.8
progress:  0.9


In [53]:
rdata_ner_df.to_csv('rdata_ner.csv')
irdata_ner_df.to_csv('irdata_ner.csv')

In [ ]:
def ner_extractor(data, entity, column_name):
    '''
    Input: dataframe, entity from spaCy, column name from dataset
    Output: list of words that are matched to the given entity
    '''
    words_names = []
    for idx, line in data.iterrows():
        if (idx % 2000 == 0):
            print("index: {}".format(idx))
        doc = nlp(line[column_name])
        names = [ent.text for ent in doc.ents if ent.label_ == entity]
        words_names.append(names)
    return words_names

def top_x_common_entity(words_names, x):
    '''
    Input: list of words that are matched to the given entity, integer to specify top x number of words
    '''
    names = [line for line in words_names for line in set(line)]
    names_count = Counter(names).most_common(x)
    print(pd.DataFrame(names_count))

In [ ]:
top_x_common_entity(ner_extractor(irdata_json, 'ORG', 'content'), 10)